In [41]:
from Crypto.Cipher import Blowfish, PKCS1_OAEP, AES
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad
from binascii import hexlify , unhexlify

plaintext = input('Enter ciphertext: ').encode()
print("Length of Ciphertext: "+str(len(plaintext.decode())))

# Dictionary of Keys
keys_iv = {}

# Blowfish Layer 1

blowfish_key = 'my 16bit Blowfish key'.encode()
blowfish_cipher = Blowfish.new(blowfish_key, Blowfish.MODE_CBC)

blowfish_ciphertext = blowfish_cipher.encrypt(pad(plaintext, Blowfish.block_size ))

keys_iv['blowfish_iv'] = hexlify(blowfish_cipher.iv)
keys_iv['blowfish_key'] = hexlify(blowfish_key)

# RSA Layer 2

rsa_key = RSA.generate(2048)
rsa_private_key = rsa_key
rsa_public_key = rsa_key.publickey()

cipher_rsa = PKCS1_OAEP.new(rsa_public_key)
rsa_plaintext = blowfish_ciphertext

rsa_ciphertext = bytearray()
for i in range(0, len(rsa_plaintext), 190):
    rsa_ciphertext.extend(cipher_rsa.encrypt(rsa_plaintext[i:i+190]))

keys_iv['rsa_n'] = rsa_private_key.n
keys_iv['rsa_e'] = rsa_private_key.e
keys_iv['rsa_d'] = rsa_private_key.d

# AES Layer 3
aes_key = 'my 16bit AES key'.encode()
aes_cipher = AES.new(aes_key, AES.MODE_CBC)
aes_plaintext = rsa_ciphertext

aes_ciphertext = aes_cipher.encrypt(pad(aes_plaintext, AES.block_size))

ciphertext = aes_ciphertext

keys_iv['aes_iv'] = hexlify(aes_cipher.iv)
keys_iv['aes_key'] = hexlify(aes_key)

'''print("Ciphertext: " + ciphertext.hex())'''
print("Length of Ciphertext(HEX):"+str(len(ciphertext.hex())))
print("Length of Ciphertext:"+str(len(ciphertext)))
print("Keys & IV: ")

for keys, values in keys_iv.items():
    print(keys, " : " , values)


Enter ciphertext: tanmoy
Length of Ciphertext: 6
Length of Ciphertext(HEX):544
Length of Ciphertext:272
Keys & IV: 
blowfish_iv  :  b'6cf04b520532ec5a'
blowfish_key  :  b'6d7920313662697420426c6f7766697368206b6579'
rsa_n  :  19493018637786835182157516892729744580290057898185767279402433020884096050910095248683715784227833540169961460628803714885853165979030340937235492206412235508633833594232723838809834569801357726075100474205859250957549694369703137789592086817167634387505377953111788444414906773642391339468330136444111550734960781832414397763712666762113471303366906645033705683879216138316140742807284485156598505200926259141839042362326489725634350778426976460498519354073015655034803778296502616992403869461213268986024337370324122680421079956342063178118656407187750371667924039729893717498290898980813029893921934433953505380129
rsa_e  :  65537
rsa_d  :  27480049314816450287311487949086792220776026507368098004852080302721846195944027038556670297767818801231407286685310209779269328849

In [ ]:
from stegano import lsb

password_cipher = AES.new(input('Enter a Password: ').encode() , AES.MODE_CBC)

for keys, values in keys_iv.items():
    aes_ciphertext = password_cipher.encrypt(pad(values, AES.block_size))

for text in texts:
    secret = lsb.hide("./badge.png", text)
    secret.save("./badge.png")

In [6]:
#Initializations

decryption_key_aes = unhexlify(keys_iv['aes_key'])
decryption_iv_aes = unhexlify(keys_iv['aes_iv'])
decryption_key_rsa = RSA.construct(rsa_components = (keys_iv['rsa_n'] , keys_iv['rsa_e'] , keys_iv['rsa_d']))
decryption_iv_blowfish = unhexlify(keys_iv['blowfish_iv'])
decryption_key_blowfish = unhexlify(keys_iv['blowfish_key'])


aes_cipher_decryption = AES.new(decryption_key_aes, AES.MODE_CBC, iv=decryption_iv_aes)
rsa_cipher_decryption = PKCS1_OAEP.new(decryption_key_rsa)
blowfish_cipher_decryption = Blowfish.new(decryption_key_blowfish, Blowfish.MODE_CBC, iv=decryption_iv_blowfish)

# AES DECRYPTION
ciphertext_rsa = unpad(aes_cipher_decryption.decrypt(ciphertext), AES.block_size)
# RSA DECRYPTION
ciphertext_blowfish = bytearray()
for i in range(0, len(ciphertext_rsa),256):
    ciphertext_rsa_segment = ciphertext_rsa[i:i+256]
    ciphertext_blowfish.extend(rsa_cipher_decryption.decrypt(ciphertext_rsa_segment))
    
# BLOWFISH DECRYPTION
decrypted_plaintext = unpad(blowfish_cipher_decryption.decrypt(ciphertext_blowfish), Blowfish.block_size)

print(decrypted_plaintext.decode())

vvyubh


In [8]:
from stegano import lsb
clear_message = lsb.reveal("./crypted-badge.png")
print(clear_message)

tanmoy is a good boy 1236


In [14]:
from Crypto.Cipher import Blowfish, PKCS1_OAEP, AES
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad
from binascii import hexlify , unhexlify
import hashlib
from datetime import datetime

plaintext = input('Enter Plaintext: ').encode()
print("Length of plaibtext: "+str(len(plaintext.decode())))

# Dictionary of Keys
password = input('Enter a Password: ')
print("Password Length: ", len(password))

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

hash = hashlib.sha1()
hash.update(password.encode())
key_iv_string = ""
password_cipher = AES.new( hash.hexdigest()[:16].encode() , AES.MODE_CBC)

# Blowfish Layer 1

blowfish_key = 'my 16bit Blowfish key'.encode()
blowfish_cipher = Blowfish.new(blowfish_key, Blowfish.MODE_CBC)

blowfish_ciphertext = blowfish_cipher.encrypt(pad(plaintext, Blowfish.block_size ))
print(blowfish_cipher.iv.hex())
key_iv_string = key_iv_string + str(hexlify(password_cipher.encrypt(pad(blowfish_cipher.iv, AES.block_size))).decode())
key_iv_string = key_iv_string + "\n"+str(hexlify(password_cipher.encrypt(pad(blowfish_key, AES.block_size))).decode())

# RSA Layer 2

rsa_key = RSA.generate(2048)
rsa_private_key = rsa_key
rsa_public_key = rsa_key.publickey()

cipher_rsa = PKCS1_OAEP.new(rsa_public_key)
rsa_plaintext = blowfish_ciphertext

rsa_ciphertext = bytearray()
for i in range(0, len(rsa_plaintext), 190):
    rsa_ciphertext.extend(cipher_rsa.encrypt(rsa_plaintext[i:i+190]))

key_iv_string = key_iv_string + "\n"+str(rsa_private_key.n)
key_iv_string = key_iv_string + "\n"+str(rsa_private_key.e)
key_iv_string = key_iv_string + "\n"+str(rsa_private_key.d)

# AES Layer 3
aes_key = 'my 16bit AES key'.encode()
aes_cipher = AES.new(aes_key, AES.MODE_CBC)
aes_plaintext = rsa_ciphertext

aes_ciphertext = aes_cipher.encrypt(pad(aes_plaintext, AES.block_size))

ciphertext = aes_ciphertext

key_iv_string = key_iv_string + "\n"+str(hexlify(password_cipher.encrypt(pad(aes_cipher.iv, AES.block_size))).decode())
key_iv_string = key_iv_string + "\n"+str(hexlify(password_cipher.encrypt(pad(aes_key, AES.block_size))).decode())


'''print("Ciphertext: " + ciphertext.hex())'''
print("Length of Ciphertext(HEX):"+str(len(ciphertext.hex())))
print("Length of Ciphertext:"+str(len(ciphertext)))
print("Keys & IV: ")

print(key_iv_string)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("\nEnd Time =", current_time)

Enter Plaintext: tanmoy
Length of plaibtext: 6
Enter a Password: tsg
Password Length:  3
Start Time = 22:33:31
dbd1f28758f9ed62
Length of Ciphertext(HEX):544
Length of Ciphertext:272
Keys & IV: 
ff88a7cb2fe8fc70f457e117d7fb7c09
48d069b38696411926a1d5d7ab09aa8289a51a412e6f357acc67e25d518e8492
20384587474050441492035193520483236261738215668761694792966669282192630005473550744137218764808549069599280414102551133575344275523043032753113369634980778741757630478137893373907102030675957039584246113180558681832732273709236324317038260750905348956848095810640478254758627565252591852838979539840890840136922148788140367439132017576454070861445678719568513101466692536930275562796077498460955692966524665921986354656951925224207047191333696251563276329438540571707279464809702633036950516235087323901482413015873305090313992926637940475597470906499775034610962897469641143670998253278429537039850348125544984586487
65537
54114630342177637224533972697768793884633333262907360118411601411658662562404265658

In [8]:
# Dictionary of Keys
password = input('Enter a Password: ')
print("Password Length: ", len(password))

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

hash = hashlib.sha1()
hash.update(password.encode())
keys_iv = {}
password_cipher = AES.new( hash.hexdigest()[:16].encode() , AES.MODE_CBC)

#Initializations

decryption_key_aes = unhexlify(keys_iv['aes_key'])
decryption_iv_aes = unhexlify(keys_iv['aes_iv'])
decryption_key_rsa = RSA.construct(rsa_components = (keys_iv['rsa_n'] , keys_iv['rsa_e'] , keys_iv['rsa_d']))
decryption_iv_blowfish = unhexlify(keys_iv['blowfish_iv'])
decryption_key_blowfish = unhexlify(keys_iv['blowfish_key'])


aes_cipher_decryption = AES.new(decryption_key_aes, AES.MODE_CBC, iv=decryption_iv_aes)
rsa_cipher_decryption = PKCS1_OAEP.new(decryption_key_rsa)
blowfish_cipher_decryption = Blowfish.new(decryption_key_blowfish, Blowfish.MODE_CBC, iv=decryption_iv_blowfish)

# AES DECRYPTION
ciphertext_rsa = unpad(aes_cipher_decryption.decrypt(ciphertext), AES.block_size)
# RSA DECRYPTION
ciphertext_blowfish = bytearray()
for i in range(0, len(ciphertext_rsa),256):
    ciphertext_rsa_segment = ciphertext_rsa[i:i+256]
    ciphertext_blowfish.extend(rsa_cipher_decryption.decrypt(ciphertext_rsa_segment))
    
# BLOWFISH DECRYPTION
decrypted_plaintext = unpad(blowfish_cipher_decryption.decrypt(ciphertext_blowfish), Blowfish.block_size)

print(decrypted_plaintext.decode())

Enter a Password: tanmoy
Password Length:  6
Start Time = 22:13:47


KeyError: 'aes_key'

Enter Plaintext: t
Length of plaintext: 1
Enter a Password: o
Password Length:  1
Start Time = 23:35:25
Length of Ciphertext(HEX):544
Length of Ciphertext:272
Keys & IV: 
7b6600d26caa7640
26814a9bb50f4d5e2f1746cd356b2e8d9c734b586f41f5bb91f01e6b829a672c
20260309445994194682152695933088601091853258337809785246240414364841245291832932103044929562822733121403454007787019541164746162038858546160234530638026798141640911526593053324614245522934157656802197755560141405809801877393973256996384429884536920306156163152521312513937832149453217751267685367131526066001739994418674450927423996180685791008431655081480177038510857830211125935629180299490599021495767936036089183887987069355058264408264624034970728152260856275630768306786787646746578415001440807696088498968961641208392018226163135795982002567597984247083990681683492304899506497803651998511999583029702428700547
65537
569565194062891256578697941424270849316118271534869590272263598055808326985870487001998538635344973112497118634463048394676

Enter a Password: o
Password Length:  1
Start Time = 23:35:44
{'aes_key': b'\xa8*;6\xdc\xdc\xe0\x9d\x9c\t\x04\x88\xc0\xe2*$', 'aes_iv': b'Ud?\x9d\xc5c)E\xbd\xc8Wg\rD\x19l', 'rsa_d': 5695651940628912565786979414242708493161182715348695902722635980558083269858704870019985386353449731124971186344630483946767219881958738643150601835222938629500772906388000891903701107993025629322729014883806784879377905444353011076207130875577280310673682803943614473607132147054733720636523417367765327067317481737707955156536884539721147202451132959307098019977791767568890520774305363858496371711605087582472285495741461890388222375640290632143175199930832606047015506849719058945512055192844567107553306120434441204606856500258498670871614988439968931061717821525709218529650628826369962236332667492895470040385, 'rsa_e': 65537, 'rsa_n': 2026030944599419468215269593308860109185325833780978524624041436484124529183293210304492956282273312140345400778701954116474616203885854616023453063802679814164091152659305332

ValueError: PKCS#7 padding is incorrect.

In [44]:
print(hexlify(b'\xa8*;6\xdc\xdc\xe0\x9d\x9c\t\x04\x88\xc0\xe2*$').decode())

a82a3b36dcdce09d9c090488c0e22a24
